In [1]:
from bs4 import BeautifulSoup as bs
import glob, os, re, csv
from datetime import datetime

In [3]:
startTime = datetime.now()

In [57]:
# parse for inventor details (by two or more inventors get a row per inventor)

def parse(xmlfile):

    def iferror(something):

        try:
            something
        except AttributeError:
            return ""


    def iferror1(tagname):

        try:
            return soup.find(tagname).text.encode('utf-8')
        except AttributeError:
            return ""

    def iferror1get(tagname,attrib):

        try:
            return soup.find(tagname).get(attrib)
        except AttributeError:
            return ""
     
    def iferror1attrib(tagname,attrib,value):

        try:
            return soup.find(tagname, attrs={attrib: value}).text.encode('utf-8')
        except AttributeError:
            return ""
    


    def iferror2(tagname,tagname2):

        try:
            return soup.find(tagname).find(tagname2).text.encode("utf-8")
        except AttributeError:
            return ""

    def iferror2num(tagname,tagname2):

        try:
            return float(soup.find(tagname).find(tagname2).text.replace(",",".").replace(" ",""))
        except (AttributeError, ValueError):
            return ""

    def iferror2get(tagname,tagname2,attrib):

        try:
            return soup.find(tagname).find(tagname2).get(attrib)
        except AttributeError:
            return ""

    def makedate(expression):
        
        try:
            pd = datetime.strptime(expression, "%Y%m%d").date()
            return pd.strftime('%Y-%m-%d')
        except:
            return ''


    soup = bs(xmlfile)

    app_land = iferror2("applicant","country")
    
    #publication attrs
    ID = iferror1get("bibliographic-data","id")
    url = re.sub("P$","","https://register.epo.org/application?number="+ID)
    title = iferror1attrib("invention-title", "lang", "en")
    status = iferror1get("bibliographic-data","status")
    public_date = makedate(iferror2("publication-reference","date"))
    public_number = iferror2("publication-reference","doc-number")
    appl_date = makedate(iferror2("application-reference","date"))
    appl_number = iferror2("application-reference","doc-number")
    event_date = makedate(iferror2("events-data","date")) #most recent
    event_descr = iferror2("events-data","event-text")

    #applicant attrs
    app_name = iferror2("applicant","name")
    app_straat = iferror2("applicant","address-1")
    app_stad = iferror2("applicant","address-2")

    #other applicants in the patent?
    if soup.find("applicant", {"sequence" : "2"}) != None:
        others = "Y"
    else:
        others = "N"

    #citations
    citations = []

    try:
        for cit in soup.find("references-cited").findAll("citation"):
        
            try:
                cit_nr = cit.find("doc-number").text
            except AttributeError:
                cit_nr = cit.get("id")
 
            citations.append(cit_nr)

            citations_num = len(citations)
            citations_nrs = ";".join(citations)
 
    except AttributeError:
     
        citations_num = 0           

    #oponents
    if soup.find("opponent") != None:
        oponent = "Y"
    else:
        oponent = "N"

    lapses = []
    lapses_nr = None
    if soup.find("term-of-grant") != None:
        for lapse in soup.find("term-of-grant").findAll("lapsed-in-country"):
            lapses.append(lapse.find("date").text)
        
        lapses_nr = len(lapses)

    row = []

    # find 
    if soup.find("inventors") != None:
    
        for ca in soup.find("inventors"):

            contract = []    
            soup = ca
            
            naam = iferror1("name")
            straat = iferror1("address-1")
            pscenstad = iferror1("address-2")
            land = iferror1("country")
             
            if naam == '':
                continue
            else: 
                contract.extend([ID,url,title,status,public_date,public_number,appl_date,appl_number,event_date,event_descr,app_name,app_straat,app_stad,app_land,others,citations_num,oponent,lapses_nr,naam,straat,pscenstad,land])
                row.append(contract)

    else:
        contract = []
        naam = "geen"
        straat = "geen"
        pscenstad = "geen"
        land = "geen"
        
        contract.extend([ID,url,title,status,public_date,public_number,appl_date,appl_number,event_date,event_descr,app_name,app_straat,app_stad,app_land,others,citations_num,oponent,lapses_nr,naam,straat,pscenstad,land])
        row.append(contract)
    
    #print row   
    return row

In [58]:
directory = glob.glob('*.xml')

in case of hiccups

In [59]:
parsed = []

In [60]:
new = [f for f in directory if f not in parsed]

In [62]:
with open("patenten.csv", "w",  newline='') as outputfile:
    writer = csv.writer(outputfile)
    header = ["ID","url","title","status","publication_date","publication_number",
              "applicant_name","applicant_street","applicant_city","applicant_land",
              "applicants_from","inventors_from","representatives_from",
              "application_date","application_number","last_event_date","last_event_description",
              "number_of_citations","oponents","nr_of_lapses"]

    writer.writerow(header)
    
    for f in new:

        filename = open(f).read()
        row = parse(filename)
        
        if row == None:
            continue
        else:
            writer.writerows(row)
            
        parsed.append(f)

/home/zufanka/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/zufanka/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
